
# Ran gather with decontaminated db. 
- Found 19 spp in 99% of genomes. 
- Extracted marker genes with checkM 
- How many of the metaGs have a signal of the  marker genes
- predict CDS using prokka: How many metaGs have the CDS signal
- manysearch x pangenomes from GTDB
- manysearch x pangenomes from GTDB+MAGs


In [1]:
# imports
import pandas as pd


In [13]:
# open df 
dfgather = pd.read_csv('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/csv_files/251201_99core.10000kb.csv', names=["query_name", "gather_GTDBMAG"], header=0)
dfgather["query_name"] = dfgather["query_name"].str.replace(r"^.*s__\s*", "", regex=True)
dfgather["query_name"] = dfgather["query_name"].str.replace("_", " ")
ids = pd.read_csv("/group/ctbrowngrp2/amhorst/2025-pig-corespp/resources/metag-wort-hq.3217.txt", header=None)[0]



In [2]:
def open_df(path_csv, contain_val, sig_type):
    df = pd.read_csv(path_csv)
    df = df[['match_name', 'query_name', 'max_containment']]
    df = df[df["max_containment"] >= contain_val]
    df = df[df["match_name"].isin(ids)]
    df["query_name"] = df["query_name"].str.replace(r".*/", "", regex=True)
    df["query_name"] = df["query_name"].str.split(".").str[0]
    df["query_name"] = df["query_name"].str.replace("_", " ")
    counts = df["query_name"].value_counts().rename(sig_type)
    return df, counts

In [12]:
# set a containment threshold if wanted
containment_threshold = 0.001
df_marker, c_marker = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/checkm/251204_manysearch.3216.marker.csv', containment_threshold, "marker_gene")
df_cds, c_cds = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/prokka/251204_manysearch.3216.CDS.csv', containment_threshold, "CDS")
df_ms_gtdb, c_gtdb = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/251204_manysearch.3216.gtdb.csv',containment_threshold, "GTDB")
df_ms_gtdmag, c_gtdb_mag = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/pangenome/251204_manysearch.3216.gtdb+mags.csv',containment_threshold, "MAG+GTDB")

In [11]:
containment_threshold = 0.001
df_isolate, c_isolate = open_df('/group/ctbrowngrp2/amhorst/2025-pig-corespp/results/isolate_genomes/isolate_genomes.manysearch.3216.csv', containment_threshold, "isolate")


In [14]:
# megre al info
df_counts = pd.concat([c_marker, c_cds, c_gtdb, c_gtdb_mag, c_isolate], axis=1).fillna(0).astype(int)
df_counts = df_counts.reset_index().rename(columns={"index": "microbe"})

df = dfgather.merge(
    df_counts,on="query_name"
)


In [15]:
df

,query_name,gather_GTDBMAG,marker_gene,CDS,GTDB,MAG+GTDB,isolate
0,Phascolarctobacterium A succinatutens,3189,3187,3212,3212,3212,3212
1,Cryptobacteroides sp900546925,3190,3176,3207,3209,3210,0
2,Oscillibacter segnis,3183,3194,3201,3203,3203,0
3,JALFVM01 sp022787145,3187,3133,3212,3200,3212,0
4,Bariatricus sp004560705,3199,3176,3210,3211,3212,0
5,Gemmiger qucibialis,3185,3195,3209,3208,3211,3211
6,Mogibacterium A kristiansenii,3197,3209,3213,3201,3216,3216
7,Phocaeicola vulgatus,3193,3208,3214,3215,3215,0
8,UBA2868 sp004552595,3204,3192,3212,3204,3212,0
9,Holdemanella porci,3189,3190,3211,3210,3212,3211


In [16]:
# reorder columns
df = df[["query_name","GTDB", "MAG+GTDB", "CDS", "marker_gene", "gather_GTDBMAG"]]
# sort by species
df = df.sort_values("query_name")

In [17]:
# add % after value
dfp = df.assign(**{
    col: df[col].apply(lambda v: f"{v} ({v/3216*100:.1f}%)")
    for col in df.columns
    if col != "query_name"
})


In [18]:
dfp

,query_name,GTDB,MAG+GTDB,CDS,marker_gene,gather_GTDBMAG
4,Bariatricus sp004560705,3211 (99.8%),3212 (99.9%),3210 (99.8%),3176 (98.8%),3199 (99.5%)
13,Butyricicoccus A intestinisimiae,3206 (99.7%),3208 (99.8%),3206 (99.7%),3187 (99.1%),3183 (99.0%)
1,Cryptobacteroides sp900546925,3209 (99.8%),3210 (99.8%),3207 (99.7%),3176 (98.8%),3190 (99.2%)
10,Fimisoma sp002320005,3211 (99.8%),3211 (99.8%),3211 (99.8%),3191 (99.2%),3188 (99.1%)
5,Gemmiger qucibialis,3208 (99.8%),3211 (99.8%),3209 (99.8%),3195 (99.3%),3185 (99.0%)
9,Holdemanella porci,3210 (99.8%),3212 (99.9%),3211 (99.8%),3190 (99.2%),3189 (99.2%)
12,JAFBIX01 sp021531895,3210 (99.8%),3216 (100.0%),3216 (100.0%),3151 (98.0%),3198 (99.4%)
3,JALFVM01 sp022787145,3200 (99.5%),3212 (99.9%),3212 (99.9%),3133 (97.4%),3187 (99.1%)
11,Lactobacillus amylovorus,3212 (99.9%),3212 (99.9%),3212 (99.9%),3212 (99.9%),3210 (99.8%)
6,Mogibacterium A kristiansenii,3201 (99.5%),3216 (100.0%),3213 (99.9%),3209 (99.8%),3197 (99.4%)
